In [93]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from _spotify.spotipy_auth import spotipy_auth


def get_artists_appeared_in_global_top_50() -> list:
    spotipy_auth()
    sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
    playlist_tracks = sp.playlist_tracks(playlist_id="37i9dQZEVXbMDoHDwVN2tF", limit=50)

    track_artists = [
        item["track"]["album"]["artists"] for item in playlist_tracks["items"]
    ]
    artists_appeared = []
    for artists in track_artists:
        for artist in artists:
            artists_appeared.append(artist)

    return artists_appeared

In [94]:
get_artists_appeared_in_global_top_50()

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/2wY79sveU1sp5g7SokKOiI'},
  'href': 'https://api.spotify.com/v1/artists/2wY79sveU1sp5g7SokKOiI',
  'id': '2wY79sveU1sp5g7SokKOiI',
  'name': 'Sam Smith',
  'type': 'artist',
  'uri': 'spotify:artist:2wY79sveU1sp5g7SokKOiI'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/3Xt3RrJMFv5SZkCfUE8C1J'},
  'href': 'https://api.spotify.com/v1/artists/3Xt3RrJMFv5SZkCfUE8C1J',
  'id': '3Xt3RrJMFv5SZkCfUE8C1J',
  'name': 'Kim Petras',
  'type': 'artist',
  'uri': 'spotify:artist:3Xt3RrJMFv5SZkCfUE8C1J'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
  'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
  'id': '06HL4z0CvFAxyc27GXpf02',
  'name': 'Taylor Swift',
  'type': 'artist',
  'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
  'href': 'https://api.spotify.com/v1/a